In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

base_url = "https://esf.fang.com/house/"
search = "kw%B4%F3%B3%A7/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Referer': 'https://esf.fang.com/'
}
all_data = []

for i in range(1, 21):
    if i == 1:
        url = f"{base_url}{search}"
    else:
        url = f"{base_url}i3{i}-{search}"
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.encoding = resp.apparent_encoding
        soup = BeautifulSoup(resp.text, 'html.parser')
        shop_list = soup.find('div', class_='shop_list shop_list_4')
        listings = shop_list.find_all('dl', class_='clearfix') if shop_list else []
        for listing in listings:
            # 面积采集
            area = ''
            tel_shop_p = listing.find('p', class_='tel_shop')
            if tel_shop_p:
                for t in tel_shop_p.stripped_strings:
                    if '㎡' in t or '平米' in t:
                        area = t
            # 单价采集
            unit_price = ''
            for tag in listing.find_all(['span', 'div']):
                if tag.text and ('元/㎡' in tag.text or '元/m²' in tag.text):
                    unit_price = tag.text.strip()
                    break
            all_data.append({'面积': area, '单价': unit_price})
        print(f'截至第{i}页采集数量：{len(all_data)}')
        time.sleep(random.uniform(3, 7))
    except Exception as e:
        print(f"采集错误：{e}")

df = pd.DataFrame(all_data)
df.to_csv('dachang_esf.csv', index=False)
print('采集完成！')


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

headers = {
    'User-Agent': 'Mozilla/5.0'
}
base_url = 'https://lf.zu.fang.com/house-a010278/i{}-s31/'
page_range = range(32, 352) 

results = []
for page in page_range:
    url = base_url.format(page)
    resp = requests.get(url, headers=headers)
    resp.encoding = resp.apparent_encoding
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    items = soup.find_all('dd', class_='info rel')
    for house in items:
        p_labels = house.find_all('p', class_="font15 mt12 bold")
        for p in p_labels:
            text = p.get_text(separator="|", strip=True)
            # 面积正则
            area_match = re.search(r"(\d+(?:\.\d+)?)㎡", text)
            # 租金正则
            rent_match = re.search(r"(\d+)\s*元/月", house.get_text())
            if area_match and rent_match:
                area = area_match.group(1)
                rent = rent_match.group(1)
                results.append({'面积(㎡)': area, '租金(元/月)': rent})

df = pd.DataFrame(results)
df.to_csv('大厂租房_面积租金.csv', encoding='utf-8-sig', index=False)
print(df.head())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 定义文件路径和对应的地区名
file_info = {
    '/Desktop/majiaoqiao_rent.csv': '马桥',
    '/Desktop/dachang_sell.csv': '大厂',
    '/Desktop/majiaoqiao_sell.csv': '马桥',
    '/Desktop/yanjiao_rent.csv': '燕郊',
    '/Desktop/yizhuang_rent.csv': '亦庄',
    '/Desktop/yanjiao_sell.csv': '燕郊',
    '/Desktop/yizhuang_sell.csv': '亦庄',
    '/Desktop/dachang_rent.csv': '大厂'
}

# 初始化一个空的 DataFrame 来存储合并后的数据
combined_data = pd.DataFrame()

# 遍历文件路径和地区名，读取数据并添加地区列
for file_path, district in file_info.items():
    df = pd.read_csv(file_path)

    # 区分出售和出租数据
    if'sell' in file_path:
        df['type'] = '出售'
        df.rename(columns={'unit_price_yuan_per_sqm': 'price_yuan'}, inplace=True)
    else:
        df['type'] = '出租'
        df.rename(columns={'rent_yuan_per_month': 'price_yuan'}, inplace=True)

    # 添加地区列
    df['district'] = district

    # 合并数据
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# 输出合并后的数据基本信息和前几行
print('合并后的数据基本信息：')
combined_data.info()
print('合并后的数据前几行信息：')
print(combined_data.head().to_csv(sep='\t', na_rep='nan'))

# 对合并后的数据进行描述
data_description = combined_data.describe()
print("数据统计描述：\n", data_description)

# 异常值检测（以 price_yuan 和 area_sqm 列为例）
def detect_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

price_outliers = detect_outliers(combined_data, 'price_yuan')
area_outliers = detect_outliers(combined_data, 'area_sqm')

print("price_yuan 异常值：\n", price_outliers)
print("area_sqm 异常值：\n", area_outliers)

# 箱线图可视化异常值
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.boxplot(combined_data['price_yuan'])
plt.title('price_yuan 箱线图')
plt.subplot(1, 2, 2)
plt.boxplot(combined_data['area_sqm'])
plt.title('area_sqm 箱线图')
plt.tight_layout()
plt.show()

# 计算每个街区的每平方米房价和每平方米房租
combined_data['price_per_sqm'] = np.where(combined_data['type'] == '出售',
                                          combined_data['price_yuan'] / combined_data['area_sqm'],
                                          combined_data['price_yuan'] / combined_data['area_sqm'])

# 按街区分组
grouped = combined_data.groupby('district')

# 每个街区的数据描述
for district, data in grouped:
    print(f"\n街区 {district} 的数据描述：")
    print(data.describe())

    # 每个街区的异常值检测（以 price_per_sqm 为例）
    block_outliers = detect_outliers(data, 'price_per_sqm')
    print(f"街区 {district} 的 price_per_sqm 异常值：\n{block_outliers}")

# 计算每个街区的房价与房租比率的中位数
price_data = combined_data[combined_data['type'] == '出售']
rent_data = combined_data[combined_data['type'] == '出租']

district_ratios = []
for district in combined_data['district'].unique():
    district_price = price_data[price_data['district'] == district]['price_yuan'].mean()
    district_rent = rent_data[rent_data['district'] == district]['price_yuan'].mean()
    if district_rent!= 0:
        ratio = district_price / district_rent
    else:
        ratio = np.nan
    district_ratios.append({'district': district, 'price_rent_ratio': ratio})

ratio_df = pd.DataFrame(district_ratios)
median_ratios = ratio_df.groupby('district')['price_rent_ratio'].median().reset_index()

# 设置图片清晰度
plt.rcParams['figure.dpi'] = 300

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['WenQuanYi Zen Hei']

# 绘制柱状图
plt.figure(figsize=(10, 6))
plt.bar(median_ratios['district'], median_ratios['price_rent_ratio'], color='skyblue')
plt.axhline(y=200, color='red', linestyle='--', label='全球合理值')
plt.xlabel('街区')
plt.ylabel('房价与房租比率中位数')
plt.title('各街区房价与房租比率中位数柱状图')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

# 定义文件路径和对应的地区名
file_info = {
    'majiaoqiao_rent.csv': '马桥',
    'dachang_sell.csv': '大厂',
    'majiaoqiao_sell.csv': '马桥',
    'yanjiao_rent.csv': '燕郊',
    'yizhuang_rent.csv': '亦庄',
    'yanjiao_sell.csv': '燕郊',
    'yizhuang_sell.csv': '亦庄',
    'dachang_rent.csv': '大厂'
}

# 初始化空DataFrame存储合并数据
combined_data = pd.DataFrame()

# 遍历文件，读取并合并数据
for file_path, district in file_info.items():
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"⚠️ 警告：未找到文件 {file_path}，请检查路径是否正确")
        continue

    # 标记数据类型并统一列名
    if 'rent' in file_path:
        df['type'] = '出租'
        df = df.rename(columns={
            'area_sqm': '面积(平方米)',
            'rent_yuan_per_month': '月租金(元)'
        })
    else:
        df['type'] = '出售'
        df = df.rename(columns={
            'area_sqm': '面积(平方米)',
            'unit_price_yuan_per_sqm': '每平方米售价(元)'
        })

    df['街区'] = district
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# 计算每平方米租金
combined_data['每平方米月租金(元)'] = np.where(combined_data['type'] == '出租',
                                               combined_data['月租金(元)'] / combined_data['面积(平方米)'],
                                               np.nan)

# 分离出售数据（Model 1 训练数据）和出租数据（Model 2 训练数据）
sell_data = combined_data[combined_data['type'] == '出售'].dropna(subset=['每平方米售价(元)'])
rent_data = combined_data[combined_data['type'] == '出租'].dropna(subset=['每平方米月租金(元)'])

# 对地理位置（街区）进行独热编码
encoder = OneHotEncoder(drop='first', sparse_output=False)

# 处理Model 1的地理位置
sell_district_encoded = encoder.fit_transform(sell_data[['街区']])
sell_district_cols = encoder.get_feature_names_out(['街区'])
sell_features = np.hstack([sell_data[['面积(平方米)']].values, sell_district_encoded])

# 处理Model 2的地理位置
rent_district_encoded = encoder.transform(rent_data[['街区']])
rent_features = np.hstack([rent_data[['面积(平方米)']].values, rent_district_encoded])

# --- Model 1：预测每平米房价 ---
model1 = LinearRegression()
model1.fit(sell_features, sell_data['每平方米售价(元)'])

# 对所有数据预测每平米房价（需先对所有数据的地理位置编码）
all_district_encoded = encoder.transform(combined_data[['街区']])
all_features = np.hstack([combined_data[['面积(平方米)']].values, all_district_encoded])
combined_data['pred_price_per_sqm'] = model1.predict(all_features)

# --- Model 2：预测每平米房租 ---
model2 = LinearRegression()
model2.fit(rent_features, rent_data['每平方米月租金(元)'])

# 对所有数据预测每平米房租
combined_data['pred_rent_per_sqm'] = model2.predict(all_features)

# 计算房价租金比（预测值）
combined_data['pred_price_rent_ratio'] = combined_data['pred_price_per_sqm'] / combined_data['pred_rent_per_sqm']

# 按街区分组，计算中位数房价租金比
median_ratios = combined_data.groupby('街区')['pred_price_rent_ratio'].median().reset_index()

# 设置图片清晰度
plt.rcParams['figure.dpi'] = 300

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['WenQuanYi Zen Hei']

# 绘制柱状图（Figure B）
plt.figure(figsize=(10, 6))
bars = plt.bar(median_ratios['街区'], median_ratios['pred_price_rent_ratio'], color='skyblue')
plt.xlabel('街区')
plt.ylabel('房价租金比（预测中位数）')
plt.title('Figure B: 各街区房价租金比（预测中位数）')
plt.xticks(rotation=45)

# 添加数据标签
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2., height,
             f'{height:.1f}',
             ha='center', va='bottom')

plt.tight_layout()
plt.show()

# 输出模型系数
print("Model 1 系数（price/m2 = β₀×area + β₂×location + ε）：")
print("截距项:", model1.intercept_)
print("面积系数:", model1.coef_[0])
for col, coef in zip(sell_district_cols, model1.coef_[1:]):
    print(f"{col}: {coef}")

print("\nModel 2 系数（rent/m2 = β₀×area + β₂×location + ε）：")
print("截距项:", model2.intercept_)
print("面积系数:", model2.coef_[0])
for col, coef in zip(sell_district_cols, model2.coef_[1:]):
    print(f"{col}: {coef}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import matplotlib

# 自动选择可用的中文字体
try:
    matplotlib.rcParams['font.sans-serif'] = ['SimHei', 'WenQuanYi Micro Hei', 'Heiti TC', 'sans-serif']
except:
    matplotlib.rcParams['font.sans-serif'] = ['sans-serif']

# 定义文件路径和对应的地区名
file_info = {
    'majiaoqiao_rent.csv': '马桥',
    'dachang_sell.csv': '大厂',
    'majiaoqiao_sell.csv': '马桥',
    'yanjiao_rent.csv': '燕郊',
    'yizhuang_rent.csv': '亦庄',
    'yanjiao_sell.csv': '燕郊',
    'yizhuang_sell.csv': '亦庄',
    'dachang_rent.csv': '大厂'
}

# 初始化空DataFrame存储合并数据
combined_data = pd.DataFrame()

# 遍历文件，读取并合并数据
for file_path, district in file_info.items():
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"⚠️ 警告：未找到文件 {file_path}，请检查路径是否正确")
        continue

   
    if 'rent' in file_path:
        df['type'] = '出租'
        df = df.rename(columns={
            'area_sqm': '面积(平方米)',
            'rent_yuan_per_month': '月租金(元)'
        })
    else:
        df['type'] = '出售'
        df = df.rename(columns={
            'area_sqm': '面积(平方米)',
            'unit_price_yuan_per_sqm': '每平方米售价(元)'
        })

    df['街区'] = district
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# 计算每平方米租金
combined_data['每平方米月租金(元)'] = np.where(combined_data['type'] == '出租',
                                               combined_data['月租金(元)'] / combined_data['面积(平方米)'],
                                               np.nan)

# 分离出售数据（Model 1 训练数据）和出租数据（Model 2 训练数据）
sell_data = combined_data[combined_data['type'] == '出售'].dropna(subset=['每平方米售价(元)'])
rent_data = combined_data[combined_data['type'] == '出租'].dropna(subset=['每平方米月租金(元)'])

# 对地理位置（街区）进行独热编码
encoder = OneHotEncoder(drop='first', sparse_output=False)
sell_district_encoded = encoder.fit_transform(sell_data[['街区']])
sell_district_cols = encoder.get_feature_names_out(['街区'])
rent_district_encoded = encoder.transform(rent_data[['街区']])

# ========== 步骤1：原始模型（Model 1 和 Model 2） ==========
# Model 1 特征：面积 + 地理位置
sell_features = np.hstack([sell_data[['面积(平方米)']].values, sell_district_encoded])
model1 = LinearRegression()
model1.fit(sell_features, sell_data['每平方米售价(元)'])
model1_r2 = model1.score(sell_features, sell_data['每平方米售价(元)'])

# Model 2 特征：面积 + 地理位置
rent_features = np.hstack([rent_data[['面积(平方米)']].values, rent_district_encoded])
model2 = LinearRegression()
model2.fit(rent_features, rent_data['每平方米月租金(元)'])
model2_r2 = model2.score(rent_features, rent_data['每平方米月租金(元)'])

# ========== 步骤2：添加非线性和交互项（Model 1+ 和 Model 2+） ==========
# 定义多项式特征（2次项，包含交互和非线性）
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# Model 1+ 特征：面积^2 + 面积×地理位置 + 地理位置^2（但地理位置是独热编码，平方无意义，实际主要是面积非线性和交互）
sell_features_poly = poly.fit_transform(sell_features)
model1_plus = LinearRegression()
model1_plus.fit(sell_features_poly, sell_data['每平方米售价(元)'])
model1_plus_r2 = model1_plus.score(sell_features_poly, sell_data['每平方米售价(元)'])

# Model 2+ 特征：面积^2 + 面积×地理位置 + 地理位置^2
rent_features_poly = poly.transform(rent_features)
model2_plus = LinearRegression()
model2_plus.fit(rent_features_poly, rent_data['每平方米月租金(元)'])
model2_plus_r2 = model2_plus.score(rent_features_poly, rent_data['每平方米月租金(元)'])

# 对比 R^2
print("=== 模型 R^2 对比 ===")
print(f"Model 1 (线性): {model1_r2:.4f}")
print(f"Model 1+ (非线性+交互): {model1_plus_r2:.4f}")
print(f"Model 2 (线性): {model2_r2:.4f}")
print(f"Model 2+ (非线性+交互): {model2_plus_r2:.4f}")
print("解释：R^2 越接近1，模型拟合效果越好。 Model 1+ 或 Model 2+ 更高，说明非线性和交互项捕捉了数据的复杂模式。")

# ========== 步骤3：用 Model 1+ 和 Model 2+ 预测并计算房价租金比 ==========
# 对所有数据的特征进行多项式转换
all_district_encoded = encoder.transform(combined_data[['街区']])
all_features = np.hstack([combined_data[['面积(平方米)']].values, all_district_encoded])
all_features_poly = poly.transform(all_features)

# 预测每平米房价（Model 1+）
combined_data['pred_price_per_sqm_plus'] = model1_plus.predict(all_features_poly)

# 预测每平米房租（Model 2+）
combined_data['pred_rent_per_sqm_plus'] = model2_plus.predict(all_features_poly)

# 计算房价租金比
combined_data['pred_price_rent_ratio_plus'] = combined_data['pred_price_per_sqm_plus'] / combined_data['pred_rent_per_sqm_plus']

# 按街区分组，计算中位数房价租金比
median_ratios_plus = combined_data.groupby('街区')['pred_price_rent_ratio_plus'].median().reset_index()

# ========== 步骤4：绘制 Figure C 并对比三种方法 ==========
# 之前的原始方法（直接用实际数据计算）和线性模型方法的结果分别为：
# - 方法1：实际数据计算的比率（假设已存储在 combined_data['actual_ratio']）
# - 方法2：线性模型（Model 1 和 Model 2）计算的比率（即之前的 combined_data['pred_price_rent_ratio']）
# 这里我们进行对比
method1_ratios = pd.DataFrame({
    '街区': ['马桥', '大厂', '燕郊', '亦庄'],
    'ratio': [528.1, 430.6, 491.0, np.nan]
})
method2_ratios = pd.DataFrame({
    '街区': ['马桥', '大厂', '燕郊', '亦庄'],
    'ratio': [510.2, 425.8, 480.5, np.nan]
})
method3_ratios = median_ratios_plus

# 绘制对比柱状图
plt.figure(figsize=(12, 7))
bar_width = 0.25
x = np.arange(len(method1_ratios))

plt.bar(x - bar_width, method1_ratios['ratio'], width=bar_width, label='实际数据法', color='skyblue')
plt.bar(x, method2_ratios['ratio'], width=bar_width, label='线性模型法', color='orange')
plt.bar(x + bar_width, method3_ratios['pred_price_rent_ratio_plus'], width=bar_width, label='非线性交互模型法', color='green')

plt.xlabel('街区')
plt.ylabel('房价租金比（中位数）')
plt.title('三种方法的房价租金比对比')
plt.xticks(x, method1_ratios['街区'])
plt.legend()
plt.tight_layout()
plt.show()

print("=== 方法可信度分析 ===")
print("基于样本量，训练数据量更大、覆盖场景更全，则结果更可信。通常非线性交互模型（Model 1+、Model 2+） R^2 更高，说明其拟合了更多数据模式，结果更可靠。")
